In [ ]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import Model, Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from keras.models import load_model
from sklearn.decomposition import PCA

In [ ]:
# Function to perform onehot encoding of labels
def onehotEncodeLabels(trainY, testY):
  # integer encode
  label_encoder = LabelEncoder()
  int_trainY = label_encoder.fit_transform(trainY.reshape(-1))
  int_testY = label_encoder.transform(testY.reshape(-1))

  # binary encode
  onehot_encoder = OneHotEncoder(sparse=False)
  int_trainY = int_trainY.reshape(len(int_trainY), 1)
  int_testY = int_testY.reshape(len(int_testY), 1)

  trainYEncoded = onehot_encoder.fit_transform(int_trainY)
  testYEncoded = onehot_encoder.transform(int_testY)
  return trainYEncoded, testYEncoded

**ResNet50 Model - With PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained ResNet50 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_features_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_classLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/ResNet50_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**ResNet50 Model - With Noise Augmentation and PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained ResNet50 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet50_augmentFeatures_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet50_augmentLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet50_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**ResNet101 Model - With PCA (0.98)**

**ResNet101 Model - With Noise Augmentation and PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained ResNet101 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet101_augmentFeatures_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet50_augmentLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/ResNet101_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**VGG16 Model - With PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained VGG16 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/VGG16_features_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_classLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/VGG16_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**VGG16 Model - With Noise Augmentation and PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained VGG16 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/VGG16_augmentFeatures_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet50_augmentLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/VGG16_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**DenseNet121 Model - With PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained DenseNet121 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/DenseNet121_features_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features/ResNet50_classLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Batch Data 64/Deep Features_CV/DenseNet121_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels

**DenseNet121 Model - With Noise Augmentation and PCA (0.98)**

In [ ]:
# Loading Deep Features extracted using pretrained DenseNet121 and labels
visFeatures = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/DenseNet121_augmentFeatures_128.npy")
visLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment/ResNet50_augmentLabels_128.npy")

In [ ]:
# Initialize StratifiedKFold using 5-fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)

# Splitting and Saving data and labels of each fold into 5 separate files
i = 0
for trainId, testId in kfold.split(visFeatures, visLabels):
  trainVisData, testVisData = visFeatures[trainId], visFeatures[testId]
  trainVisLabels, testVisLabels = visLabels[trainId], visLabels[testId]
  trainVisLabels, testVisLabels = onehotEncodeLabels(trainVisLabels, testVisLabels)
  
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/trainFeatures_128_" + str(i) + ".npy", trainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/testFeatures_128_" + str(i) + ".npy", testVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/testLabels_128_" + str(i) + ".npy", testVisLabels)

  i += 1

In [ ]:
# Performing PCA to each fold data and saving PCA applied data into separate files
for i in range(0, 5):
  # Loading training and testing data of each fold
  trainVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/trainFeatures_128_" + str(i) + ".npy")
  trainVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/trainLabels_128_" + str(i) + ".npy")
  testVisData = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/testFeatures_128_" + str(i) + ".npy")
  testVisLabels = np.load("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121/testLabels_128_" + str(i) + ".npy")

  # Initialize PCA with 0.98 variance and applied to training and testing data
  pca = PCA(0.98)
  pcatrainVisData = pca.fit_transform(trainVisData)
  del trainVisData

  pcatestVisData = pca.transform(testVisData)
  del testVisData
  
  # Saving PCA applied training and testing data fold wise into separate file
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121_PCA/trainFeatures_128_" + str(i) + ".npy", pcatrainVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121_PCA/trainLabels_128_" + str(i) + ".npy", trainVisLabels)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121_PCA/testFeatures_128_" + str(i) + ".npy", pcatestVisData)
  np.save("/content/drive/MyDrive/Colab Notebooks/Fall 2021/Final Project/Visual Features_abs_Overlap_RGB/Data Augmentation/Deep Features_Augment_CV/DenseNet121_PCA/testLabels_128_" + str(i) + ".npy", testVisLabels)

  del pcatrainVisData, pcatestVisData
  del trainVisLabels, testVisLabels